# Street Colouring with OSMnx
This was inspired by previous work by Erin Davis and Cedric Scherer.

* [The beautiful hidden logic of cities](https://erdavis.com/2019/07/27/the-beautiful-hidden-logic-of-cities/)
* [#Berlin road network by name/suffix forming colorful networks](https://twitter.com/CedScherer/status/1195420409434382336)
* [Erin's code](https://github.com/erdavis1/RoadColors)
* [Cedric's code](https://github.com/Z3tt/30DayMapChallenge)

Their code is very interesting in that it shows how to manipulate shapefiles in R in order to extract the data and chart. However, R is not my first language (I speak Python slightly better), and I'm a notorious big fan of OSMnx, so I wanted to see if this could be replicated with my favourite set of tools.

**TL;DR**: it can, and it only takes a few lines of code.

For comparison and reference, Cedric's version is the one I aimed to replicate directly:
![Berlin](images/BERLINORIGINAL.png)


In [ ]:
import osmnx as ox
import pandas as pd
import geopandas as gpd
import networkx as nx
%matplotlib inline
ox.config(log_console=True, use_cache=True)

# Colouring
All I do here:
* define a set of colours in three language-specific dictionaries
* use a function to generate an array of colours while enumerating the "edges" (roads or parts of roads)

Note the caveat on how to query Nominatim (OpenStreetMap search engine) for the location. The result must be a POLYGON or POLYLINE. If a POINT is returned, you'd better check for multiple results and selecting one that isn't.

In [ ]:
dict_ENG = {'row' :'#c60000','close':'#ee9999','gate': '#ec0b88','wynd':'#e18a1e','brae':'#ff8888','den': '#dd3333','lane' :'#de8140','place' :'#a64dff','road': '#019868','street' : '#f6cf71','way': '#0000c6','avenue': '#651eac','drive': '#1e8ae1','terrace':'#009999','crescent': '#003d99','circle':'#33adff'}
dict_GER = {'straße': '#f6cf71','weg':'#019868','allee':'#ec0b88','damm' :'#651eac','platz' :'#e18a1e','chaussee' :'#9dd292','see' :'#2b7de5','ufer' :'#2b7de5','steg' :'#2b7de5'}
dict_FRA = {'rue' :'#019868','place' : '#f6cf71','avenue' : '#ec0b88','boulevard' : '#651eac','passage' : '#e18a1e','pont' : '#9dd292','quai' : '#2b7de5'}


In [ ]:
# This function takes the road name and a 3-letter code for the language and it returns the colour
def colourcode(road_name, language):
    ret_val = "#c6c6c6"
    if (language=='GER'):
        for key in dict_GER: 
            if road_name.lower().endswith(key):
                ret_val =  dict_GER.get(key)
    elif (language=='ENG'):
        for key in dict_ENG: 
            if road_name.lower().endswith(key):
                ret_val =  dict_ENG.get(key) 
    elif (language=='FRA'):
        for key in dict_FRA: 
            if road_name.lower().startswith(key): #note difference from ENG / GER
                ret_val =  dict_FRA.get(key)
    return ret_val

In [ ]:
# Set place and language; the place is basically a Nominatim query. It must return a POLYGON/POLYLINE, not a POINT, so you might have to play with it a little, or set which_result below accordingly    
place='Berlin, Germany'
language='GER'

# note the which_result parameter, as per comment above
G = ox.graph_from_place(place, network_type='all', which_result=2) 

'''
place='Aberdeen, United Kingdom'
language='ENG'

# note the which_result parameter, as per comment above
G = ox.graph_from_place(place, network_type='all', which_result=1) #note Aberdeen needs which_result to be 1
'''


In [ ]:
# For the colouring, we take the attributes from each edge found extract the road name, and use the function above to create the colour array
edge_attributes = ox.graph_to_gdfs(G, nodes=False)
ec = [colourcode(str(row['name']).lower(), language) for index, row in edge_attributes.iterrows()]

# We can finally draw the plot
fig, ax = ox.plot_graph(G, bgcolor='white', axis_off=True, node_size=0, node_color='w', node_edgecolor='gray', node_zorder=2,
                        edge_color=ec, edge_linewidth=0.5, edge_alpha=1, fig_height=20, dpi=300)

In [ ]:
# Appendix 
# Are you curious about other feature of the streets? With this piece of code, you can see what other elements you could colour..

edge_attributes = ox.graph_to_gdfs(G, nodes=False)
edge_attributes.head() 